In [199]:
import pandas as pd
import numpy as np
import json

### Files locations

In [31]:
FILE_USERS = "../../data/raw_users.csv"
FILE_USERS_TWEETS = "../../data/raw_tweets.csv"
FILE_RETWEETERS = "../../data/retweets.csv"
FILE_RETWEETERS_USERS = "../../data/retweets_users.csv"
POPULARITY = "../../data/popularity_v2.csv"

### Load Datasets

In [32]:
df_users = pd.read_csv(filepath_or_buffer=FILE_USERS, sep=",")
df_tweets = pd.read_csv(filepath_or_buffer=FILE_USERS_TWEETS, sep=",")

# df_users = df_users[:100]
# df_tweets = df_tweets[:100]

/var/folders/7r/g2lhjv3x7bb_lgcnhj7_q9hr0000gn/T/ipykernel_21207/1493569851.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_users = pd.read_csv(filepath_or_buffer=FILE_USERS, sep=",")


In [33]:
df_users

,Unnamed: 0,user_id,name,username,followers,following,tweet_count,verified,created_at
0,0,874735375,ARobles,anthonysradical,278,203,3844,False,2012-10-12 01:55:26+00:00
1,1,2331261888,James Smith Jr,iamjamesnumber1,2292,4490,96610,False,2014-02-07 04:06:30+00:00
2,2,1043073024092766208,"STOP SAYING POC, IT’s A CUSS WORD",ADOSALLY1,1076,1190,13969,False,2018-09-21 09:42:43+00:00
3,3,1350969232897552385,Pure Rap Podcast,PureRapPodcast,2465,2111,27373,False,2021-01-18 00:52:30+00:00
4,4,811604288443187200,April Showers ⛈🌪💫,moonlitefire,1570,874,27642,False,2016-12-21 16:08:36+00:00
...,...,...,...,...,...,...,...,...,...
1072382,1072382,608420462,1056050,Skeeter_Bentley,388,0,5846,False,2012-06-14 19:30:38
1072383,1072383,786792941658382339,1056051,billybr61,503,504,32684,False,2016-10-14 04:57:10
1072384,1072384,497947909,1056052,BobGoble,2754,3176,117664,False,2012-02-20 14:46:48
1072385,1072385,50706690,1056053,511NY,3579,193,5447942,False,2009-06-25 17:22:38


In [34]:
df_tweets

,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets
0,0,1473095873357746185,Bullshit ass holding call right now bro agains...,874735375,2021-12-21 00:59:59+00:00,Original Tweet,0,0,0,0,iPhone,en,"['3', '6', '11', '26', '27', '28']","['TV Shows', 'Sports Event', 'Sport', 'Sports ...",NaN
1,1,1473095873118806021,Me at parties when my social meter has reached...,2331261888,2021-12-21 00:59:59+00:00,Original Tweet,5,0,4,1,iPhone,en,[],[],NaN
2,2,1473095869780180994,❤️❤️❤️❤️❤️❤️❤️Oh My!!!! Today is a good day!!!...,1043073024092766208,2021-12-21 00:59:58+00:00,Original Tweet,9,0,3,0,iPhone,en,['123'],['Ongoing News Story'],NaN
3,3,1473095868546879489,Giveaway entry has ended! 🚨 https://t.co/WcVKk...,1350969232897552385,2021-12-21 00:59:58+00:00,Quote Tweet,3,0,1,0,iPhone,en,"['46', '47']","['Brand Category', 'Brand']","[ReferencedTweet(id: 1468043149188632583, type..."
4,4,1473095868534431744,Heard it was @MillerLite Monday.. drink respon...,811604288443187200,2021-12-21 00:59:58+00:00,Original Tweet,35,10,0,0,Android,en,"['45', '46', '47', '65', '66', '67']","['Brand Vertical', 'Brand Category', 'Brand', ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072382,1072382,1223002761480101890,"Yep, that’s usually how it goes😂 https://t.co/...",608420462,2020-01-30 21:59:15,Original Tweet,1,0,0,0,iPhone,en,[],[],NaN
1072383,1072383,1223002761387814912,If you don't know Adam ant Dirty bag Schiff-le...,786792941658382339,2020-01-30 21:59:15,Quote Tweet,0,0,1,0,Android,en,"['10', '35']","['Person', 'Politician']","[ReferencedTweet(id: 1223000963356790784, type..."
1072384,1072384,1223002759601016834,"Make no mistake, because of republican rules a...",497947909,2020-01-30 21:59:14,Original Tweet,0,0,0,0,Android,en,"['45', '46', '47']","['Brand Vertical', 'Brand Category', 'Brand']",NaN
1072385,1072385,1223002759147982851,Cleared: Incident on #I287 NB at North of Exit...,50706690,2020-01-30 21:59:14,Original Tweet,0,0,0,0,NaN,en,[],[],NaN


### Sort tweets by timestamp

In [35]:
df_tweets = df_tweets.sort_values(by='timestamp', ascending=True).reset_index()
df_tweets.head(5)

,index,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets
0,1028700,1028700,1212176422770544641,2️⃣0️⃣1️⃣9️⃣ WHAT. A. YEAR. 🌟 #bestnine2019 #t...,29885607,2020-01-01 00:59:14,Original Tweet,35,0,1,0,iPhone,en,[],[],NaN
1,1028701,1028701,1212176421659062274,Gold N Hot 🤪 https://t.co/VeO2k4Mq4h,1130217871286636545,2020-01-01 00:59:14,Quote Tweet,0,0,0,0,iPhone,en,[],[],"[ReferencedTweet(id: 1212167212120920072, type..."
2,1028698,1028698,1212176423189909506,and he still cheating https://t.co/zYAQ8wc6Vg,3322006549,2020-01-01 00:59:15,Quote Tweet,2,2,1,7,iPhone,en,"['10', '60']","['Person', 'Athlete']","[ReferencedTweet(id: 1211878384961228802, type..."
3,1028697,1028697,1212176425916280833,Got my hot coco bc it cold out siiiiide boii h...,18707915,2020-01-01 00:59:15,Original Tweet,1,1,0,0,iPhone,en,[],[],NaN
4,1028696,1028696,1212176426088181760,Disgruntled Pope Francis pulls himself free fr...,858762062,2020-01-01 00:59:15,Original Tweet,0,0,0,0,Android,en,"['10', '46', '47']","['Person', 'Brand Category', 'Brand']",NaN


### Delete duplicate users

In [36]:
df_users.shape

(1072387, 9)

In [37]:
df_users = df_users.drop_duplicates(subset=['user_id'], keep="first")
df_users

,Unnamed: 0,user_id,name,username,followers,following,tweet_count,verified,created_at
0,0,874735375,ARobles,anthonysradical,278,203,3844,False,2012-10-12 01:55:26+00:00
1,1,2331261888,James Smith Jr,iamjamesnumber1,2292,4490,96610,False,2014-02-07 04:06:30+00:00
2,2,1043073024092766208,"STOP SAYING POC, IT’s A CUSS WORD",ADOSALLY1,1076,1190,13969,False,2018-09-21 09:42:43+00:00
3,3,1350969232897552385,Pure Rap Podcast,PureRapPodcast,2465,2111,27373,False,2021-01-18 00:52:30+00:00
4,4,811604288443187200,April Showers ⛈🌪💫,moonlitefire,1570,874,27642,False,2016-12-21 16:08:36+00:00
...,...,...,...,...,...,...,...,...,...
1072375,1072375,324616347,1056043,TwitTitty10,388,251,38688,False,2011-06-26 23:10:32
1072376,1072376,249105428,1056044,gwarmothwftv,10598,189,17492,True,2011-02-08 11:19:53
1072381,1072381,395058107,1056049,sgeezy99,572,773,6236,False,2011-10-21 02:06:15
1072382,1072382,608420462,1056050,Skeeter_Bentley,388,0,5846,False,2012-06-14 19:30:38


## Calculate tweet reach

#### These values indicate the maximum possible of people that could have seen a given tweet

To do this, a sum of all the original tweet user's followers is made with the followers from all the users that retweeted/quoted the tweet

Note: this assumes that if a retweet is then shared by retweet again, those next audiences are not taken into account

In [133]:
df_retweeters = pd.read_csv(filepath_or_buffer=FILE_RETWEETERS_USERS, sep=",")
df_retweets = pd.read_csv(filepath_or_buffer=FILE_RETWEETERS, sep=",")

In [134]:
df_retweets.head()

,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets
0,0,1475758320568971271,RT @Keiththebat: Zack Snyder's back must be so...,3457095793,2021-12-28 09:19:36,Retweet Tweet,0,0,4,0,Android,en,[],[],"[ReferencedTweet(id: 1473095800364351490, type..."
1,1,1473588933883801601,RT @Keiththebat: Zack Snyder's back must be so...,1206541010408333313,2021-12-22 09:39:14,Retweet Tweet,0,0,4,0,Bot,en,[],[],"[ReferencedTweet(id: 1473095800364351490, type..."
2,2,1473588921749573635,RT @Keiththebat: Zack Snyder's back must be so...,1369228958739263488,2021-12-22 09:39:11,Retweet Tweet,0,0,4,0,Android,en,[],[],"[ReferencedTweet(id: 1473095800364351490, type..."
3,3,1473430854680838144,RT @gameallniteshow: Getting in some Coffee Tr...,3223132542,2021-12-21 23:11:05,Retweet Tweet,0,0,2,0,Android,en,[],[],"[ReferencedTweet(id: 1473095644709535745, type..."
4,4,1473324608963100679,RT @Tails7354: Sonic X: FIU Eye-Catch Card: Fl...,1256240643258671105,2021-12-21 16:08:54,Retweet Tweet,0,0,2,0,iPhone,en,[],[],"[ReferencedTweet(id: 1473200836071329793, type..."


In [135]:
df_retweeters.head()

,Unnamed: 0,user_id,name,username,followers,following,tweet_count,verified,created_at
0,0,3457095793,SALEH ALOTAIBI,SALEHAL49202807,29,0,7473,False,2015-09-05 07:49:00
1,1,1206541010408333313,DC Tweets,RetweetDc,7851,40,406311,False,2019-12-16 11:47:16
2,2,1369228958739263488,Cottencandy,Cottenc57931689,100,309,30127,False,2021-03-09 10:10:16
3,3,3223132542,アマノ,sikkoukakari,3383,4707,10649,False,2015-05-22 10:48:05
4,4,1256240643258671105,💙Sonic The Dragonhog🌙 (On hiatus),SDragonhog,657,1017,121701,False,2020-05-01 15:15:05


### Removing duplicated retweeters

37432 retweeters will be removed

In [136]:
df_retweeters.shape

(278349, 9)

In [137]:
df_retweeters = df_retweeters.drop_duplicates(subset=['user_id'], keep="first")
df_retweeters

,Unnamed: 0,user_id,name,username,followers,following,tweet_count,verified,created_at
0,0,3457095793,SALEH ALOTAIBI,SALEHAL49202807,29,0,7473,False,2015-09-05 07:49:00
1,1,1206541010408333313,DC Tweets,RetweetDc,7851,40,406311,False,2019-12-16 11:47:16
2,2,1369228958739263488,Cottencandy,Cottenc57931689,100,309,30127,False,2021-03-09 10:10:16
3,3,3223132542,アマノ,sikkoukakari,3383,4707,10649,False,2015-05-22 10:48:05
4,4,1256240643258671105,💙Sonic The Dragonhog🌙 (On hiatus),SDragonhog,657,1017,121701,False,2020-05-01 15:15:05
...,...,...,...,...,...,...,...,...,...
278344,278344,3736161195,‏َ,noahsoIo,4020,290,29116,False,2015-09-22 06:42:48
278345,278345,1220520282017976320,Ju Quer Abraçar O Josh🇨🇦♥️,uniter_juju,11,79,57,False,2020-01-24 01:35:01
278346,278346,42664231,Gilbert Economic Dev,GilbertAZEcoDev,3215,557,3657,True,2009-05-26 16:51:49
278347,278347,86356349,"Gilbert, Arizona",GilbertYourTown,25992,2666,22969,True,2009-10-30 17:18:49


In [138]:
df_retweets.shape

(278349, 15)

In [139]:
df_retweets = df_retweets.sort_values(by='timestamp', ascending=True).reset_index()
df_retweets.head(5)

,index,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets
0,267872,267872,1212176450679377921,This is the way and only way Jay Armstrong Cut...,833382201785978882,2020-01-01 00:59:21,Quote Tweet,1,0,1,0,Android,en,[],[],"[ReferencedTweet(id: 1212175890785996801, type..."
1,267871,267871,1212176611627458560,RT @LexingtonPD: TRAFFIC ALERT - A collision o...,3028245479,2020-01-01 01:00:00,Retweet Tweet,0,0,4,0,iPhone,en,[],[],"[ReferencedTweet(id: 1212176549769830401, type..."
2,267870,267870,1212176685002571776,RT @alsilicaniyours: This is the way and only ...,833382201785978882,2020-01-01 01:00:17,Retweet Tweet,0,0,1,0,Android,en,[],[],"[ReferencedTweet(id: 1212176450679377921, type..."
3,267869,267869,1212176700861227009,Let’s not https://t.co/gMH6UWVB6q,27785930,2020-01-01 01:00:21,Quote Tweet,0,3,0,0,iPhone,en,[],[],"[ReferencedTweet(id: 1212176423189909506, type..."
4,267868,267868,1212176835603288064,RT @LexingtonPD: TRAFFIC ALERT - A collision o...,1012475880248143873,2020-01-01 01:00:53,Retweet Tweet,0,0,4,0,Android,en,[],[],"[ReferencedTweet(id: 1212176549769830401, type..."


### Merge retweets datasets

In [140]:
df_retweets_info = pd.merge(df_retweets[['tweet_id', 'user_id', 'topics', 'referenced_tweets']], df_retweeters[['user_id', 'followers', 'following']], how='left', on="user_id")
df_retweets_info.head(5)

,tweet_id,user_id,topics,referenced_tweets,followers,following
0,1212176450679377921,833382201785978882,[],"[ReferencedTweet(id: 1212175890785996801, type...",20,622
1,1212176611627458560,3028245479,[],"[ReferencedTweet(id: 1212176549769830401, type...",11059,4125
2,1212176685002571776,833382201785978882,[],"[ReferencedTweet(id: 1212176450679377921, type...",20,622
3,1212176700861227009,27785930,[],"[ReferencedTweet(id: 1212176423189909506, type...",10230,6114
4,1212176835603288064,1012475880248143873,[],"[ReferencedTweet(id: 1212176549769830401, type...",9685,10633


### Removing non-direct retweets

When colecting the retweets some tweets that mencioned the url of the original tweet and because of that were also collected as "retweets".
Since this tipe of tweets don't count to the retweet count of the original tweet and they only represent 0,0012% of the retweets collected they will be removed

In [141]:
df_retweets_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278349 entries, 0 to 278348
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   tweet_id           278349 non-null  int64 
 1   user_id            278349 non-null  int64 
 2   topics             278349 non-null  object
 3   referenced_tweets  278019 non-null  object
 4   followers          278349 non-null  int64 
 5   following          278349 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 14.9+ MB


In [142]:
df_retweets_info[pd.isna(df_retweets_info['referenced_tweets'])]

,tweet_id,user_id,topics,referenced_tweets,followers,following
465,1213173157735141376,786792941658382339,[],NaN,503,504
635,1213652399782604806,876320380792844292,[],NaN,539,398
645,1213731859131064320,3315742299,[],NaN,326,464
685,1213908969183424513,3215222674,[],NaN,405,446
2325,1216434627549847555,1089894708208373761,[],NaN,1017,1244
...,...,...,...,...,...,...
277556,1489616172404420614,838245040858222592,[],NaN,1214,2244
277584,1490216196716875780,1456396368361279492,[],NaN,715,386
277593,1490692560209207298,1468588637415944192,[],NaN,109,193
277759,1492506389503254537,1328709172423249921,[],NaN,99,538


In [143]:
df_retweets[df_retweets['tweet_id'] == 1213652399782604806]['text']

635    #FlyEaglesFly #PhillyvsEveryone https://t.co/G...
Name: text, dtype: object

In [144]:
df_retweets_info = df_retweets_info[df_retweets_info['referenced_tweets'].notna()]

In [153]:
def get_ref_tweet_id(ref_tweets):
    g = str(ref_tweets)
    if isinstance(g, str) and g != 'nan' and not pd.isna(g):
        return int(g.split()[1].replace(',', ''))
    else: return None

In [154]:
df_retweets_info['ref_tweed_id'] = df_retweets_info['referenced_tweets'].apply(lambda ref_tweet: get_ref_tweet_id(ref_tweet))
df_retweets_info.head()

,tweet_id,user_id,topics,referenced_tweets,followers,following,ref_tweed_id
0,1212176450679377921,833382201785978882,[],"[ReferencedTweet(id: 1212175890785996801, type...",20,622,1212175890785996801
1,1212176611627458560,3028245479,[],"[ReferencedTweet(id: 1212176549769830401, type...",11059,4125,1212176549769830401
2,1212176685002571776,833382201785978882,[],"[ReferencedTweet(id: 1212176450679377921, type...",20,622,1212176450679377921
3,1212176700861227009,27785930,[],"[ReferencedTweet(id: 1212176423189909506, type...",10230,6114,1212176423189909506
4,1212176835603288064,1012475880248143873,[],"[ReferencedTweet(id: 1212176549769830401, type...",9685,10633,1212176549769830401


In [179]:
df_tweets['ref_tweed_id'] = df_tweets['referenced_tweets'].apply(lambda ref_tweet: get_ref_tweet_id(ref_tweet))
# converting the reference ids that exist to int
df_tweets['ref_tweed_id'] = pd.to_numeric(df_tweets['ref_tweed_id'], errors='coerce').fillna(0).astype(np.int64)
df_tweets.head()

,index,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets,ref_tweed_id
0,1028700,1028700,1212176422770544641,2️⃣0️⃣1️⃣9️⃣ WHAT. A. YEAR. 🌟 #bestnine2019 #t...,29885607,2020-01-01 00:59:14,Original Tweet,35,0,1,0,iPhone,en,[],[],NaN,0
1,1028701,1028701,1212176421659062274,Gold N Hot 🤪 https://t.co/VeO2k4Mq4h,1130217871286636545,2020-01-01 00:59:14,Quote Tweet,0,0,0,0,iPhone,en,[],[],"[ReferencedTweet(id: 1212167212120920072, type...",1212167212120920064
2,1028698,1028698,1212176423189909506,and he still cheating https://t.co/zYAQ8wc6Vg,3322006549,2020-01-01 00:59:15,Quote Tweet,2,2,1,7,iPhone,en,"['10', '60']","['Person', 'Athlete']","[ReferencedTweet(id: 1211878384961228802, type...",1211878384961228800
3,1028697,1028697,1212176425916280833,Got my hot coco bc it cold out siiiiide boii h...,18707915,2020-01-01 00:59:15,Original Tweet,1,1,0,0,iPhone,en,[],[],NaN,0
4,1028696,1028696,1212176426088181760,Disgruntled Pope Francis pulls himself free fr...,858762062,2020-01-01 00:59:15,Original Tweet,0,0,0,0,Android,en,"['10', '46', '47']","['Person', 'Brand Category', 'Brand']",NaN,0


In [195]:
def calculate_tweet_reach(tweet_id, user_id, topics, retweets_count, ref_tweed_id):
    total_reach = 0
    total_reach += df_users[df_users['user_id'] == user_id]['followers'].sum()
    if topics != '[]' and retweets_count > 0:
        if ref_tweed_id is not None and ref_tweed_id != 0:
            total_reach += df_retweets_info[(df_retweets_info['ref_tweed_id'] == tweet_id) | (df_retweets_info['ref_tweed_id'] == ref_tweed_id)]['followers'].sum()
        else:
            total_reach += df_retweets_info[df_retweets_info['ref_tweed_id'] == tweet_id]['followers'].sum()
    return total_reach

In [198]:
df_tweets['reach'] = df_tweets.apply(lambda tweet_row: calculate_tweet_reach(tweet_row['tweet_id'], tweet_row['user_id'], tweet_row['topics'], tweet_row['retweet_count'], tweet_row['ref_tweed_id']), axis=1)
df_tweets.head(5)

tweet_id 1212176423189909506
ref_tweed_id 1211878384961228800
default 653
after search 13837 

tweet_id 1212176429053550593
ref_tweed_id 0
default 2636
basic search 3322 

tweet_id 1212176436783656960
ref_tweed_id 0
default 2528
basic search 2528 

tweet_id 1212176450679377921
ref_tweed_id 1212175890785996800
default 20
after search 40 

tweet_id 1212176465560657921
ref_tweed_id 0
default 366
basic search 366 

tweet_id 1212176474158977029
ref_tweed_id 0
default 2352
basic search 2352 

tweet_id 1212176484938506240
ref_tweed_id 0
default 1856
basic search 1856 

tweet_id 1212176491649339392
ref_tweed_id 1212174934308524032
default 5516
after search 5516 

tweet_id 1212176495201927168
ref_tweed_id 0
default 137
basic search 4833 

tweet_id 1212176506060828672
ref_tweed_id 0
default 894
basic search 894 

tweet_id 1212176514311184386
ref_tweed_id 0
default 707
basic search 804 

tweet_id 1212176515590434816
ref_tweed_id 1212121574901313536
default 62081
after search 62081 

tweet_id 1212

KeyboardInterrupt: 

## Topic analysis

### Loading the previously trained topic analysis model and it's evaluations of each tweet text

In [ ]:
df_tweets.head(5)

In [ ]:
def process_topic(topics:str):
    s = topics
    for i in range(topics.count('\'')):
        s = s.replace('\'', '"')
    t = json.loads(s)
    if len(t) > 0:
        return t[0]
    return None

In [ ]:
df_tweets['topics'] = df_tweets['topics'].apply(lambda topics: process_topic(topics))
df_tweets['topics_ids'] = df_tweets['topics_ids'].apply(lambda topics: process_topic(topics))
df_tweets.head(5)

### Sentiment analysis

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_scores(sentence, prints):
    if prints: print("\nSentence:", sentence)

    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)

    if sentiment_dict['compound'] >= 0.05:
        if prints: print("Positive")
        return "Positive"
    elif sentiment_dict['compound'] <= - 0.05:
        if prints: print("Positive")
        return "Negative"
    else:
        if prints: print("Neutral")
        return "Neutral"

In [ ]:
df_tweets['sentiment'] = df_tweets['text'].apply(lambda tweet_text: sentiment_scores(tweet_text, False))

In [ ]:
df_tweets.head(5)

### Tweet popularity

To define tweet popularity, each tweet with at least 1 retweet or 1 quote tweet was considered popular (1)
if the criteria wasn't met it was marked as unpopular (0)

In [ ]:
def tweet_popularity_label(retweet_count, quote_count):
    if retweet_count > 0 or quote_count > 0:
        return 1
    else:
        return 0

In [ ]:
df_tweets['popularity'] = df_tweets.apply(lambda row: tweet_popularity_label(row['retweet_count'], row['quote_count']), axis=1)

In [ ]:
df_tweets['popularity'].value_counts()

## Combine the two datasets into the final one

For every tweet select the relevant columns and merge the corresponding information from the original user

In [ ]:
df_popularity = pd.merge(df_tweets[['tweet_id', 'text', 'timestamp', 'user_id', 'like_count', 'retweet_count', 'quote_count', 'reply_count', 'reach', 'topics_ids', 'topics', 'sentiment', 'popularity']],
                   df_users[['user_id', 'followers', 'following', 'tweet_count', 'verified', 'created_at']],
                   on="user_id")
df_popularity.head(5)

## Save the final version of the dataset

In [ ]:
df_popularity.to_csv(POPULARITY, sep=',', date_format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_popularity